# IMPORTAR LIBRERIAS

In [1]:
import pandas as pd
import os
import re

import warnings
warnings.filterwarnings('ignore')

C:\Users\OT\AppData\Local\Temp\ipykernel_15812\589167050.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


# ORIGEN DATASETS 

Los datos utilizados en este estudio provienen del Panel de Consumo Alimentario en Hogares, gestionado por el Ministerio de Agricultura, Pesca y Alimentación y elaborado con la información proporcionada por TAYLOR NELSON SOFRES S.A.U. Este panel ofrece una visión detallada de la demanda de alimentos en los hogares españoles, abarcando información sobre las compras realizadas, su costo y las características de los hogares. El panel cubre todos los hogares de la Península, Baleares y Canarias, excluyendo Ceuta y Melilla.

La fuente de datos incluye registros desde 1990 hasta 2024, tanto anuales como mensuales. En este análisis, se ha optado por examinar los datos del período de 1999 a 2024 a nivel mensual. Los datasets están disponibles en formato .xlsx y/o se presentan anualmente y con distintas hojas por mes. 

Dado que los archivos originales pueden ser grandes y pesados, se han implementado las siguientes optimizaciones:

- Unificación de datos mensuales y anuales: Se ha consolidado la información mensual y anual en un único archivo. Esta estructura facilita la gestión de la información  y permite un acceso más ágil a los datos.

- Separación de datos nacionales y por comunidad autónoma.

- Eliminación de la hoja PORTADA: Se ha eliminado la hoja PORTADA presente en todos los documentos .xlsx, ya que no aporta valor adicional.

- Conversión a formato .csv: Se ha convertido los archivos .xlsx a .csv para reducir el espacio de almacenamiento. Este formato es más ligero y permite una lectura y escritura más rápidas, optimizando el manejo de los datos para futuros procesos.

En este estudio se utilizarán exclusivamente los datos a nivel nacional. No obstante, dado que los datos nacionales están integrados en el mismo archivo que los correspondientes a las comunidades autónomas, se generará un segundo conjunto de datos segmentado por comunidad autónoma. Este segundo dataset quedará disponible para análisis futuros o con fines teóricos.

# FUNCIONES TRANSFORMACIÓN 

Se describen las funciones encargadas de procesar los archivos .xlsx provenientes de la fuente original. Estas funciones se dividen en dos grupos: aquellas que procesan los datos a nivel nacional y aquellas que lo hacen a nivel de comunidad autónoma.
En ambos casos, se desarrollan dos funciones principales: una para procesar un solo archivo y otra para procesar todos los archivos de manera conjunta. Esta separación permite reutilizar las funciones procesar_archivo y procesar_archivo_ccaa con archivos futuros, facilitando tanto la preparación de nuevos datasets como la ampliación del conjunto de datos actual

## NACIONAL

In [2]:
def procesar_archivo(file_path, anno):
    """
    Procesa un archivo Excel para extraer datos de todas las hojas, exceptuando la hoja 'Portada'.
    Los datos se filtran, se renombran las columnas y se agregan las columnas de Año y Mes.

    Parámetros:
    - file_path (str): Ruta del archivo Excel a procesar. 
    - anno (int): Año asociado a los datos en el archivo.

    Retorna:
    - pd.DataFrame: Un DataFrame con los datos filtrados, con las columnas renombradas y las nuevas columnas 'Año' y 'Mes' 
    agregadas.
    """
    datos_completos = pd.DataFrame()
    
    with pd.ExcelFile(file_path) as archivo:
        # Obtener nombres de hojas, excluyendo 'Portada'
        hojas = [hoja for hoja in archivo.sheet_names if hoja.lower() != 'portada']
        
        for mes in hojas:
            # Leer datos de la hoja del mes
            df = pd.read_excel(file_path, sheet_name=mes)
    
            # Filtrar las filas y columnas necesarias
            df_filtrado = df.iloc[2:, :7]
            
            # Renombrar las columnas del DataFrame
            df_filtrado.columns = ['Alimentos', 'CONSUMO X CAPITA', 'GASTO X CAPITA', 'PENETRACION (%)', 
                                   'PRECIO MEDIO kg ó litros', 'VALOR (Miles Euros)', 'VOLUMEN (Miles kg ó litros)']
            
            # Agregar columnas de 'Año' y 'Mes'
            df_filtrado['Año'] = anno
            df_filtrado['Mes'] = mes
            
            # Concatenar los datos de esta hoja al DataFrame completo
            datos_completos = pd.concat([datos_completos, df_filtrado], ignore_index=True)
    
    return datos_completos

def procesar_todos_los_archivos(directorio_entrada, directorio_salida):
    """
    Procesa todos los archivos Excel en el directorio de entrada para los años del 2024 al 1999.
    Combina los datos de todos los archivos en un solo DataFrame y lo guarda en un nuevo archivo .csv en el directorio 
    de salida.

    Parámetros:
    - directorio_entrada (str): Ruta del directorio que contiene los archivos Excel a procesar.
    - directorio_salida (str): Ruta del directorio donde se guardarán los archivos combinados.

    Retorna:
    - None: Guarda el DataFrame combinado en archivos CSV en el directorio de salida.
    """
    dataframes_completos = pd.DataFrame()
    
    for anno in range(2024, 1998, -1):
        file_path = os.path.join(directorio_entrada, f'{anno}-CCAA.xlsx')
        
        if os.path.isfile(file_path):
            print(f"Procesando archivo: {file_path}")
            datos = procesar_archivo(file_path, anno)
            dataframes_completos = pd.concat([dataframes_completos, datos], ignore_index=True)
        else:
            print(f"Archivo no encontrado: {file_path}")
    
    # Guardar en formato CSV
    csv_path = os.path.join(directorio_salida, 'nacional_combinados.csv')
    dataframes_completos.to_csv(csv_path, index=False)
    print(f"Datos combinados guardados en {csv_path}")

## CCAA

In [3]:
def procesar_archivo_ccaa(file_path):
    """
    Procesa el archivo Excel para extraer y combinar los datos en un DataFrame por comunidad autónoma (CCAA).
    """
    # Función para obtener el año del archivo a partir del nombre
    def obtener_anno_del_archivo(file_path):
        return int(re.search(r'\d{4}', file_path).group())

    # Función para leer las hojas del archivo Excel, excluyendo la primera hoja (Portada)
    def leer_hojas(file_path):
        with pd.ExcelFile(file_path) as archivo:
            return archivo.sheet_names[1:]  # Excluye la primera hoja

    # Función para obtener las comunidades autónomas a partir de la hoja 'enero' o 'Enero'
    def obtener_ccaa(file_path):
        df = None
        try:
            df = pd.read_excel(file_path, sheet_name='enero')
        except ValueError:
            print(f"Hoja 'enero' no encontrada en {file_path}. Intentando con 'Enero'.")
            try:
                df = pd.read_excel(file_path, sheet_name='Enero')
            except ValueError:
                print(f"Hoja 'Enero' no encontrada en {file_path}.")
        
        if df is not None:
            fila_3 = df.iloc[0]
            return fila_3[2:].dropna().tolist()  # Excluye la primera columna
        else:
            print(f"CCAA no encontradas en ninguna de las hojas esperadas para el archivo {file_path}.")
            return []

    # Función para procesar el DataFrame y estandarizar las columnas
    def procesar_dataframe(df):
        df_new = df.iloc[1:]  # Eliminar la primera fila
        df_new.columns = df_new.iloc[0]  # Asignar nombres de columna
        df_new = df_new.iloc[1:]  # Eliminar la fila de nombres de columna
        df_new.reset_index(drop=True, inplace=True)  # Restablecer índices
        return df_new

    # Función para crear un DataFrame seleccionado con columnas desplazadas y agregar columnas adicionales
    def crear_dataframe_seleccionado(df_new, inicio_columna, fin_columna, mes, anno, comunidad):
        df_seleccionado = df_new.iloc[:, inicio_columna:fin_columna].copy()  # Seleccionar columnas desplazadas
        df_seleccionado['Mes'] = mes
        df_seleccionado['Año'] = anno
        df_seleccionado['CCAA'] = comunidad
        df_seleccionado['Alimentos'] = df_new.iloc[:, 0].reset_index(drop=True)  # Agregar columna 'Alimentos'
        df_seleccionado = df_seleccionado[['Alimentos'] + [col for col in df_seleccionado.columns if col != 'Alimentos']]
        return df_seleccionado

    # Iniciar el procesamiento
    anno = obtener_anno_del_archivo(file_path)  # Obtener el año del archivo
    hojas = leer_hojas(file_path)  # Leer las hojas del archivo Excel
    ccaa = obtener_ccaa(file_path)  # Obtener las comunidades autónomas

    desplazamiento = 6  # Número de columnas por CCAA
    dataframes_ccaa = {comunidad: pd.DataFrame() for comunidad in ccaa}  # Crear diccionario para cada CCAA

    # Procesar cada mes (hoja)
    for mes in hojas:
        df = pd.read_excel(file_path, sheet_name=mes)  # Leer la hoja correspondiente al mes
        df_new = procesar_dataframe(df)  # Procesar el DataFrame para estandarizar las columnas

        # Procesar los datos para cada comunidad autónoma
        for i, comunidad in enumerate(ccaa):
            inicio_columna = 7 + i * desplazamiento
            fin_columna = inicio_columna + 6
            df_seleccionado = crear_dataframe_seleccionado(df_new, inicio_columna, fin_columna, mes, anno, comunidad)
            dataframes_ccaa[comunidad] = pd.concat([dataframes_ccaa[comunidad], df_seleccionado], ignore_index=True)
    
    return dataframes_ccaa

def procesar_todos_los_archivos_ccaa(directorio_entrada, directorio_salida):
    """
    Procesa todos los archivos Excel en el directorio de entrada y guarda los resultados en .csv en el directorio de salida.
    """
    dataframes_ccaa_completos = pd.DataFrame()  # DataFrame combinado
    
    for anno in range(2024, 1998, -1):
        file_path = os.path.join(directorio_entrada, f'{anno}-CCAA.xlsx')
        
        if os.path.isfile(file_path):
            print(f"Procesando archivo: {file_path}")
            dataframes_ccaa = procesar_archivo_ccaa(file_path)
            
            for df in dataframes_ccaa.values():
                dataframes_ccaa_completos = pd.concat([dataframes_ccaa_completos, df], ignore_index=True)
        else:
            print(f"Archivo no encontrado: {file_path}")
    
    # Guardar todos los datos combinados en un solo archivo CSV
    csv_path = os.path.join(directorio_salida, 'datos_combinados_CCAA.csv')
    dataframes_ccaa_completos.to_csv(csv_path, index=False)
    print(f"Datos combinados guardados en {csv_path}")

# CREAR DATASET

In [4]:
# Definir los directorios de entrada y salida
directorioEntrada = './Dataset/Dataset_origen'
directorioSalida = './Dataset/Dataset_final'

In [5]:
# Llamamos a la función pertinente para procesar los archivos y transformar en un dataset de datos a nivel nacional
procesar_todos_los_archivos(directorioEntrada, directorioSalida)

Procesando archivo: ./Dataset/Dataset_origen\2024-CCAA.xlsx
Procesando archivo: ./Dataset/Dataset_origen\2023-CCAA.xlsx
Procesando archivo: ./Dataset/Dataset_origen\2022-CCAA.xlsx
Procesando archivo: ./Dataset/Dataset_origen\2021-CCAA.xlsx
Procesando archivo: ./Dataset/Dataset_origen\2020-CCAA.xlsx
Procesando archivo: ./Dataset/Dataset_origen\2019-CCAA.xlsx
Procesando archivo: ./Dataset/Dataset_origen\2018-CCAA.xlsx
Procesando archivo: ./Dataset/Dataset_origen\2017-CCAA.xlsx
Procesando archivo: ./Dataset/Dataset_origen\2016-CCAA.xlsx
Procesando archivo: ./Dataset/Dataset_origen\2015-CCAA.xlsx
Procesando archivo: ./Dataset/Dataset_origen\2014-CCAA.xlsx
Procesando archivo: ./Dataset/Dataset_origen\2013-CCAA.xlsx
Procesando archivo: ./Dataset/Dataset_origen\2012-CCAA.xlsx
Procesando archivo: ./Dataset/Dataset_origen\2011-CCAA.xlsx
Procesando archivo: ./Dataset/Dataset_origen\2010-CCAA.xlsx
Procesando archivo: ./Dataset/Dataset_origen\2009-CCAA.xlsx
Procesando archivo: ./Dataset/Dataset_or

In [6]:
# Llamamos a la función pertinente para procesar los archivos y transformar en un dataset de datos a nivel de comunidad autónoma
procesar_todos_los_archivos_ccaa(directorioEntrada, directorioSalida)

Procesando archivo: ./Dataset/Dataset_origen\2024-CCAA.xlsx
Procesando archivo: ./Dataset/Dataset_origen\2023-CCAA.xlsx
Procesando archivo: ./Dataset/Dataset_origen\2022-CCAA.xlsx
Procesando archivo: ./Dataset/Dataset_origen\2021-CCAA.xlsx
Procesando archivo: ./Dataset/Dataset_origen\2020-CCAA.xlsx
Procesando archivo: ./Dataset/Dataset_origen\2019-CCAA.xlsx
Hoja 'enero' no encontrada en ./Dataset/Dataset_origen\2019-CCAA.xlsx. Intentando con 'Enero'.
Procesando archivo: ./Dataset/Dataset_origen\2018-CCAA.xlsx
Hoja 'enero' no encontrada en ./Dataset/Dataset_origen\2018-CCAA.xlsx. Intentando con 'Enero'.
Procesando archivo: ./Dataset/Dataset_origen\2017-CCAA.xlsx
Hoja 'enero' no encontrada en ./Dataset/Dataset_origen\2017-CCAA.xlsx. Intentando con 'Enero'.
Procesando archivo: ./Dataset/Dataset_origen\2016-CCAA.xlsx
Hoja 'enero' no encontrada en ./Dataset/Dataset_origen\2016-CCAA.xlsx. Intentando con 'Enero'.
Procesando archivo: ./Dataset/Dataset_origen\2015-CCAA.xlsx
Hoja 'enero' no enco